# Исследование надежности заемщиков

**Описание:**

Исследование, направленное на вычисление факторов влияния на возврат кредита в срок заёмщиками.

Результаты исследования предполагается учесть в модели кредитного скоринга.

**Цель:** 

Выяснить, существует ли зависимость между принадлежностью клиентов к определённой категории и их способностью возврата кредита в срок.

**Описание данных:**

Входные данные от банка — статистика о платёжеспособности клиентов.

- children — количество детей в семье

- days_employed — общий трудовой стаж в днях

- dob_years — возраст клиента в годах

- education — уровень образования клиента

- education_id — идентификатор уровня образования

- family_status — семейное положение

- family_status_id — идентификатор семейного положения

- gender — пол клиента

- income_type — тип занятости

- debt — имел ли задолженность по возврату кредитов

- total_income — ежемесячный доход

- purpose — цель получения кредита

**План работы:**

1. Изучить общую информацию о данных.


2. Предобработка данных:

2.1 Удаление пропусков

2.2 Обработка аномальных значений

2.3 Удаление пропусков(продолжение)

2.4 Изменение типов данных

2.5 Обработка дубликатов

2.6 Категоризация данных

3. Исследование данных и ответы на вопросы:

3.1 Зависимость между количеством детей и возвратом кредита в срок

3.2 Зависимость между семейным положением и возвратом кредита в срок

3.3 Зависимость между уровнем дохода и возвратом кредита в срок

3.4 Как разные цели кредита влияют на его возврат в срок

4. Общий вывод

### 1. Изучаем общую информацию о данных

Импортируем библиотеку и читаем файл с данными

In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/data.csv')

Изучаем первые 20 строк файла и общую информацию

In [ ]:
data.head(20)

In [ ]:
data.info()

### 2.Предобработка данных

**2.1 Удаление пропусков**

Выводим количество пропущенных значений для каждого столбца

In [ ]:
data.isna().sum()

Столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type

In [ ]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == type), 'total_income'].median()

**2.2 Обработка аномальных значений**

В столбце days_employed были обнаружены аномальные значения (отрицательное кол-во дней трудового стажа). Заменяем их с помощью метода abs()

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выводим медианное значение трудового стажа days_employed в днях

In [ ]:
data.groupby('income_type')['days_employed'].median()

Выводим перечень уникальных значений столбца children

In [ ]:
data['children'].unique()

В этом столбце есть два аномальных значения. Удаляем строки, в которых встречаются такие аномальные значения

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выводим перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены

In [ ]:
data['children'].unique()

**2.3 Удаление пропусков(продолжение)**

Заполняем пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [ ]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == type), 'days_employed'].median()

Убеждаемся, что все пропуски заполнены

In [ ]:
data.isna().sum()

**2.4 Изменение типов данных**

Заменяем вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()

In [ ]:
data['total_income'] = data['total_income'].astype('int')

**2.5 Обработка дубликатов**

Обрабатываем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведим их к нижнему регистру

In [ ]:
data['education'] = data['education'].str.lower()

Выводим количество строк-дубликатов в данных и удаляем их

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

**2.6 Категоризация данных**

Формируем диапазоны и создаём в датафрейме на их основе столбец total_income_category с категориями:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Используем собственную функцию categorize_income() и метод apply()

In [ ]:
def categorize_income(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выводим на экран перечень уникальных целей взятия кредита из столбца purpose

In [ ]:
data['purpose'].unique()

Создаём функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Используем собственную функцию categorize_purpose() и метод apply(). Изучим данные в столбце purpose и определим, какие подстроки помогут нам правильно определить категорию

In [ ]:
def categorize_purpose(purpose):
    try:
        if 'автом' in purpose:
            return 'операции с автомобилем'
        elif 'жил' in purpose or 'недвиж' in purpose:
            return 'операции с недвижимостью'
        elif 'свадьб' in purpose:
            return 'проведение свадьбы'
        elif 'образован' in purpose:
            return 'получение образования'
    except:
        return 'категория не определена'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### 3. Исследование данных и ответы на вопросы:

**3.1 Зависимость между количеством детей и возвратом кредита в срок**

Группируем данные о просрочках клиентов по количеству детей и агрегируем их по: общему количеству клиентов, количеству клиентов с просрочкой, соотношению второго к первому. Сортируем по убыванию это соотношение для наглядности результата.

In [ ]:
data.groupby('children')['debt'].agg(['count', 'sum', 'mean']) \
.rename(columns={'count': 'total_clients', 'sum': 'debtors', 'mean': 'ratio'}).sort_values(by='ratio', ascending=False)

**Вывод:** 

Исходя из данных таблицы, зависимость между количеством детей и возвратом кредита в срок присутствует. 

Общий тренд говорит о том, что с возрастанием количества детей, относительное количество клиентов с просрочками растёт. 

Количество клиентов с 5 детьми слишком мало, чтобы дать какую-то оценку по этой категории. 

Также из общего тренда выбиваются клиенты с 3 детьми, однако в целом это не сильно влияет на общую картину.

**3.2 Зависимость между семейным положением и возвратом кредита в срок**

Группируем данные о просрочках клиентов по семейному положению и агрегируем их по: общему количеству клиентов, количеству клиентов с просрочкой, соотношению второго к первому. Сортируем по убыванию это соотношение для наглядности результата.

In [ ]:
data.groupby('family_status')['debt'].agg(['count', 'sum', 'mean']) \
.rename(columns={'count': 'total_clients', 'sum': 'debtors', 'mean': 'ratio'}).sort_values(by='ratio', ascending=False)

**Вывод:**

Зависимость между семейным положением и возвратом кредита в срок присутствует.

Те клиенты, которые не были женаты/замужем, а также те клиенты, которые официально не регистрировали брак, чаще совершают просрочку по возврату кредита в относительном выражении, чем те клиенты, которые на данный момент женаты/замужем или были ранее в официально зарегистрированном браке.

**3.3 Зависимость между уровнем дохода и возвратом кредита в срок**

Группируем данные о просрочках клиентов по категориям доходов и агрегируем их по: общему количеству клиентов, количеству клиентов с просрочкой, соотношению второго к первому. Сортируем по убыванию это соотношение для наглядности результата.

In [ ]:
data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean']) \
.rename(columns={'count': 'total_clients', 'sum': 'debtors', 'mean': 'ratio'}).sort_values(by='ratio', ascending=False)

**Вывод:**

Категории по доходу:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Исходя из полученных данных, делаем вывод, что зависимости между уровнем дохода и возвратом кредита в срок нет, так как даже убрав категории E и A с небольшим количеством людей, явного тренда в ту или иную сторону нет.

**3.4 Как разные цели кредита влияют на его возврат в срок**

Группируем данные о просрочках клиентов по целям кредита и агрегируем их по: общему количеству клиентов, количеству клиентов с просрочкой, соотношению второго к первому. Сортируем по убыванию это соотношение для наглядности результата.

In [ ]:
data.groupby('purpose_category')['debt'].agg(['count', 'sum', 'mean']) \
.rename(columns={'count': 'total_clients', 'sum': 'debtors', 'mean': 'ratio'}).sort_values(by='ratio', ascending=False)

**Вывод:**

Анализ данных показал, что такие цели кредита, как операции с автомобилем и получение образования, в относительном выражении чаще сталкиваются с просрочкой клиентов, чем кредиты на проведение свадеб и на операции с недвижимостью.

### 4. Общий вывод

На основе проведённого исследования, можно заключить, что на возврат кредита в срок имеют влияние факторы: количество детей, семейное положение.

Уровень дохода не является показательным фактором влияния на возврат кредита в срок.

Кроме того, исследование по категориям целей кредита выявило закономерность: клиенты, которые берут кредит на автомобиль или получение образования, чаще не возвращают в срок, чем клиенты, которые берут кредит на проведение свадеб и операции с недвижимостью.